In [ ]:
import os
import numpy as np
from osgeo import gdal
from skimage.exposure import match_histograms

"""
This script does the following:
1) Takes a directory of KMZ files.
2) Converts each KMZ to a temporary GeoTIFF.
3) Histogram-matches each GeoTIFF to the first KMZ file (band-wise if multi-band).
4) Merges all histogram-matched GeoTIFFs into a single GeoTIFF using "max" pixel overlap.
5) Cleans up temporary files.
"""

kmz_directory = r"C:\Users\magst\Downloads\download (7)"
output_tif_path = r"C:\Users\magst\Downloads\merged_palisades_1_10_2025.tif"

# -----------------------------------------------------------------------------------------
# 1) Gather all KMZ files
# -----------------------------------------------------------------------------------------
kmz_files = [
    os.path.join(kmz_directory, f)
    for f in os.listdir(kmz_directory)
    if f.lower().endswith(".kmz")
]
if not kmz_files:
    print("No KMZ files found. Exiting.")
    raise SystemExit

# -----------------------------------------------------------------------------------------
# 2) Convert the first KMZ to a reference array (for histogram matching)
# -----------------------------------------------------------------------------------------
first_kmz_ds = gdal.Open(kmz_files[0])
if first_kmz_ds is None:
    print(f"Could not open reference KMZ: {kmz_files[0]}")
    raise SystemExit

ref_array = first_kmz_ds.ReadAsArray().astype(np.float32)
first_kmz_ds = None

# -----------------------------------------------------------------------------------------
# 3) Convert each KMZ to GeoTIFF, then histogram-match it to the reference array
# -----------------------------------------------------------------------------------------
temp_tif_list = []

for kmz_file in kmz_files:
    base, _ = os.path.splitext(kmz_file)
    temp_tif = base + "_temp.tif"

    # Convert KMZ -> GeoTIFF
    ds_kmz = gdal.Open(kmz_file)
    if ds_kmz is None:
        print(f"Could not open {kmz_file}. Skipping...")
        continue
    gdal.Translate(temp_tif, ds_kmz, format="GTiff")
    ds_kmz = None

    # Open the newly-created GeoTIFF in update mode
    ds_temp = gdal.Open(temp_tif, gdal.GA_Update)
    if ds_temp is None:
        print(f"Could not open temporary TIF {temp_tif}. Skipping...")
        continue

    arr_temp = ds_temp.ReadAsArray().astype(np.float32)

    # If single-band, just match directly
    if arr_temp.ndim == 2:
        matched = match_histograms(arr_temp, ref_array)
        ds_temp.GetRasterBand(1).WriteArray(matched)
    else:
        # Multi-band: match each band using the corresponding band of the ref array if possible
        # If the reference has fewer bands than the target, we just match to band 0.
        for b in range(arr_temp.shape[0]):
            # If reference is also multi-band, pick the matching band index.
            if ref_array.ndim == 3 and b < ref_array.shape[0]:
                ref_band = ref_array[b]
            elif ref_array.ndim == 3:
                ref_band = ref_array[0]
            else:
                ref_band = ref_array

            matched = match_histograms(arr_temp[b], ref_band)
            ds_temp.GetRasterBand(b + 1).WriteArray(matched)

    ds_temp.FlushCache()
    ds_temp = None
    temp_tif_list.append(temp_tif)

# -----------------------------------------------------------------------------------------
# 4) Merge all histogram-matched GeoTIFFs with "max" pixel overlap
# -----------------------------------------------------------------------------------------
if temp_tif_list:
    merge_options = gdal.WarpOptions(
        format="GTiff",
        resampleAlg=gdal.GRA_Max  # use "max" for overlapping areas
    )
    merged_ds = gdal.Warp(
        destNameOrDestDS=output_tif_path,
        srcDSOrSrcDSTab=temp_tif_list,
        options=merge_options
    )
    merged_ds = None
else:
    print("No valid GeoTIFFs created. Exiting.")
    raise SystemExit

# -----------------------------------------------------------------------------------------
# 5) Clean up temporary TIFF files
# -----------------------------------------------------------------------------------------
for tif in temp_tif_list:
    if os.path.exists(tif):
        os.remove(tif)

print("Merging complete. Output written to:", output_tif_path)
